In [0]:
%sql

CREATE TABLE 4_prod.pacs_ai.pacs_examcode_dict
AS
SELECT
  *,
  CONCAT(
    '{\n short code: ', short_code, ' ;\n ',
    'description: ', preferred, ' ;\n ',
    'procedure name: ', sct_fsn, ' ;\n ',
    'laterality: ', COALESCE(laterality, 'Null'), ' ;\n ',
    'region: ', COALESCE(region, 'Null'), ' ;\n ',
    'system: ', COALESCE(system, 'Null'), ' ;\n}'
  ) AS concat_text
FROM read_files(
  '/Volumes/4_prod/pacs/base/Annex-1-DID_lookup_group.csv',
  format => 'csv',
  header => true,
  mode => 'FAILFAST')

In [0]:
%sql
ALTER TABLE 4_prod.pacs_ai.pacs_examcode_dict SET TBLPROPERTIES (delta.enableChangeDataFeed = true) 

In [0]:
from databricks.vector_search.client import VectorSearchClient

client = VectorSearchClient()

index = client.create_delta_sync_index(
  endpoint_name="pacs_examcode_vector_search",
  source_table_name="4_prod.pacs_ai.pacs_examcode_dict",
  index_name="4_prod.pacs_ai.pacs_examcode_concat_vs_index",
  pipeline_type="TRIGGERED",
  primary_key="short_code",
  embedding_source_column="concat_text",
  embedding_model_endpoint_name="azure_openai_embedding_endpoint"
)